In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tfrecorder import TFrecorder
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%pylab inline

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Populating the interactive namespace from numpy and matplotlib


In [2]:
tfr = TFrecorder()
def input_fn_maker(path, data_info_path, shuffle=False, batch_size = 1, epoch = 1, padding = None):
    def input_fn():
        filenames = tfr.get_filenames(path=path, shuffle=shuffle)
        dataset=tfr.get_dataset(paths=filenames, data_info=data_info_path, shuffle = shuffle, 
                            batch_size = batch_size, epoch = epoch, padding =padding)
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()
    return input_fn

padding_info = ({'image':[28,28,1],'label':[]})
test_input_fn = input_fn_maker('mnist_tfrecord/test/',  'mnist_tfrecord/data_info.csv',batch_size = 512,
                               padding = padding_info)
train_input_fn = input_fn_maker('mnist_tfrecord/train/',  'mnist_tfrecord/data_info.csv', shuffle=True, batch_size = 128,
                               padding = padding_info)
train_eval_fn = input_fn_maker('mnist_tfrecord/train/',  'mnist_tfrecord/data_info.csv', batch_size = 512,
                               padding = padding_info)

In [8]:
def model_fn(features, mode):
    # shape: [None,28,28,1]
    # create RNN cells:
    rnn_cells = [tf.nn.rnn_cell.GRUCell(dim) for dim in [128,256]]
    # stack cells for multi-layers RNN
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_cells)
    # create RNN layers
    outputs, last_state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                   inputs=tf.reshape(features['image'],[-1,28,28]),
                                   dtype=tf.float32)
    # shape: outputs: [None,28,256]
    # shape: last_state: [None,256]
    dense1 = tf.layers.dense(inputs=last_state[1], units=1024, activation=tf.nn.relu, name= 'dense1')
    # shape: [None,1024]
    dropout = tf.layers.dropout(inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dense1, units=10, name= 'output')
    # shape: [None,10]
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "labels": features['label'],
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
        }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=features['label'], logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=features['label'], predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [9]:
rm -rf mnist_model_rnn/

In [10]:
mnist_classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir="mnist_model_rnn")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'mnist_model_rnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4ffd0d7dd8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
mnist_classifier.train(
    input_fn=train_input_fn)

read dataframe from mnist_tfrecord/data_info.csv
   default  isbyte length_type   name  shape     type
0      NaN   False       fixed  image  [784]  float32
1      NaN   False       fixed  label     []    int64
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into mnist_model_rnn/model.ckpt.
INFO:tensorflow:loss = 2.3036835, step = 1
INFO:tensorflow:global_step/sec: 13.8069
INFO:tensorflow:loss = 0.2501039, step = 101 (7.246 sec)
INFO:tensorflow:global_step/sec: 13.7133
INFO:tensorflow:loss = 0.35629043, step = 201 (7.294 sec)
INFO:tensorflow:global_step/sec: 13.6721
INFO:tensorflow:loss = 0.101596184, step = 301 (7.316 sec)
INFO:tensorflow:global_step/sec: 15.0853
INFO:tensorflow:loss = 0.17481667, step = 401 (6.626 sec)
INFO:tensorflow:Saving checkpoints for 430 into mnist_model_rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.131294.


In [ ]:
eval_results = mnist_classifier.evaluate(input_fn=train_eval_fn)
print('train set')
print(eval_results)

In [12]:
eval_results = mnist_classifier.evaluate(input_fn=test_input_fn)
print('test set')
print(eval_results)

read dataframe from mnist_tfrecord/data_info.csv
   default  isbyte length_type   name  shape     type
0      NaN   False       fixed  image  [784]  float32
1      NaN   False       fixed  label     []    int64
INFO:tensorflow:Starting evaluation at 2018-02-08-10:37:06
INFO:tensorflow:Restoring parameters from mnist_model_rnn/model.ckpt-430
INFO:tensorflow:Finished evaluation at 2018-02-08-10:37:08
INFO:tensorflow:Saving dict for global step 430: accuracy = 0.9646, global_step = 430, loss = 0.11123172
test set
{'accuracy': 0.9646, 'loss': 0.11123172, 'global_step': 430}
